In [1]:
%cd wwdrive2

/home/ubuntu/wwdrive2


In [2]:
!pip install --upgrade happytransformer transformers datasets
!pip install wandb
#!pip install numpy==1.24.3  # This is an example; choose any version within the compatible range


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://git

In [3]:
import os
os.environ["WANDB_API_KEY"] = "339e8cbc62b895edfe6f9f7476adf12a96dd3ed6"


In [4]:
import wandb
wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="t5-mini-grammar",
)

wandb: Currently logged in as: willwade. Use `wandb login --relogin` to force relogin


In [7]:
import pandas as pd

def remove_non_string_rows(df, columns):
    """
    Remove rows where any specified columns do not contain string entries.
    
    Parameters:
    - df: pandas DataFrame containing your data.
    - columns: list of column names to check for non-string entries.
    
    Returns:
    - The DataFrame with rows containing non-string entries in the specified columns removed.
    - A list of indices of the rows that were removed.
    """
    indices_to_remove = set()

    for column in columns:
        for i, entry in enumerate(df[column]):
            if not isinstance(entry, str):
                # Add the index of the non-string entry to the set of indices to remove
                indices_to_remove.add(i)
    
    # If there are any indices to remove, drop those rows
    if indices_to_remove:
        print(f"Removing rows with non-string entries in the following indices: {sorted(indices_to_remove)}")
        df = df.drop(index=list(indices_to_remove)).reset_index(drop=True)
    else:
        print("No non-string entries found in the specified columns.")
    
    return df, indices_to_remove

# Example usage
df = pd.read_csv("eval.csv")  # Load your dataset
columns_to_check = ['input', 'target']  # Specify the columns to check
df, indices_removed = remove_non_string_rows(df, columns_to_check)
# Optionally, save the cleaned DataFrame back to a CSV
df.to_csv("cleaned_eval.csv", index=False)

df = pd.read_csv("train.csv")  # Load your dataset
columns_to_check = ['input', 'target']  # Specify the columns to check
df, indices_removed = remove_non_string_rows(df, columns_to_check)
# Optionally, save the cleaned DataFrame back to a CSV
df.to_csv("cleaned_train.csv", index=False)



Removing rows with non-string entries in the following indices: [478711]
Removing rows with non-string entries in the following indices: [1872771]


In [ ]:
from happytransformer import HappyTextToText, TTEvalArgs

happy_tt = HappyTextToText("T5", "t5-small")
before_result = happy_tt.eval("cleaned_eval.csv")
print("Before loss:", before_result.loss)


02/13/2024 16:49:18 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
02/13/2024 16:49:18 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...


Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/489029 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
from happytransformer import TTTrainArgs
# Define your parameters
params = TTTrainArgs(
    fp16=True,
    learning_rate=5e-5,
    num_train_epochs=3,
    batch_size=32,
    save_steps=1000  # Automatic checkpoint every 1000 steps
)

#model.save_pretrained("my_checkpoint")
#model = HappyTextToText.from_pretrained("my_checkpoint")

# Specify your training file path
train_file_path = "cleaned_train.csv"

In [ ]:
# Train the model with Happy Transformers
happy_tt.train(train_file_path, args=params)

Generating train split: 0 examples [00:00, ? examples/s]

02/13/2024 18:59:13 - INFO - happytransformer.happy_transformer -   Tokenizing training data...


Map:   0%|          | 0/1722748 [00:00<?, ? examples/s]

Map:   0%|          | 0/191417 [00:00<?, ? examples/s]

02/13/2024 19:00:22 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0


Step,Training Loss,Validation Loss
1,3.318900,3.337981


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
from happytransformer import HappyGeneration
happy_gen = HappyGeneration(model_type="T5", model_name="willwade/t5-small-spoken-typo")
happy_gen.save("model/")

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, Qwen2Config, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, RwkvConfig, S

In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Assuming you have your model loaded
model_name = "willwade/t5-small-spoken-typo"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
#tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# Save the model and tokenizer
model_save_path = "model/t5-small-spoken-typo"
tokenizer_save_path = "model/t5-small-spoken-typo"

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

# To load the model and tokenizer later
model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_save_path)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
before_loss = happy_tt.eval("cleaned_eval.csv")
print("After loss: ", before_loss.loss)

02/13/2024 23:04:42 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...
Parameter 'function'=<function HappyTextToText._tok_function.<locals>.__preprocess_function at 0x7f8b09f07370> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/489029 [00:00<?, ? examples/s]

In [ ]:
# test it!
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)
example_2 = "grammar: I am enjoys, writtings articles ons AI."
result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

In [ ]:
#Lets try optimising it now..
from transformers import T5ForConditionalGeneration
import torch

# Load your pre-trained model
model = T5ForConditionalGeneration.from_pretrained("./model/t5-small-spoken-typo")

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the quantized model
quantized_model.save_pretrained("./model/quantized-t5-small-spoken-typo")